This modified example from the ditribution uses Queue inputstream to do a word count of words. Words are fetched from a URL and bunched in batches of n words. Each batch then is parallelized as an RDD an put into the rddQueue, which is then pushed to the inputstream for processing.

In [2]:
#import the necessary modules
import urllib2
import time
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
#
# function that returns batches of bn from the list of words
#
def getBatches(url, bn):
  webFile = urllib2.urlopen(url).read()
  words = webFile.split(",")
  #add an addtional word of the batches are not even
  if len(words) % bn != 0:
    words = words + ['jules']
  j = bn
  batches = []
  for i in range(0, len(words), bn):
      batches.append(words[i:j])
      j = j + bn
  return batches

Fetch the batches 10 words, each batch as a list of comma separated words

In [4]:
batches = getBatches('http://www.textfixer.com/resources/common-english-words.txt', 10)

create the streaming context

In [6]:
ssc = StreamingContext(sc, 1)

Create the queue through which RDDs can be pushed he QueueInputDStream

In [8]:
rddQueue = map(lambda l: ssc.sparkContext.parallelize(l), batches)
inputStream = ssc.queueStream(rddQueue)
mappedStream = inputStream.flatMap(lambda w: (w, 1))
reducedStream = mappedStream.reduceByKey(lambda a, b: a + b)
reducedStream.pprint()
ssc.start()
time.sleep(6)
ssc.stop(stopSparkContext=True, stopGraceFully=True)
